# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coquimbo,-29.9533,-71.3436,16.85,67,0,6.17,CL,1695506196
1,1,minervino murge,41.0843,16.0784,18.07,82,55,2.11,IT,1695506196
2,2,margaret river,-33.9500,115.0667,9.14,81,4,2.60,AU,1695506197
3,3,isafjordur,66.0755,-23.1240,3.68,91,98,7.20,IS,1695506197
4,4,basco,20.4487,121.9702,28.68,77,16,7.70,PH,1695506197


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot

map_plot=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df=city_data_df[(city_data_df["Max Temp"]>20) & \
                              (city_data_df["Max Temp"]<27) & (city_data_df["Wind Speed"]<4.5) & \
                              (city_data_df["Humidity"]<89) & (city_data_df["Cloudiness"]<5)]
narrowed_city_df
# Drop any rows with null value
clean_narrowed_city_df=narrowed_city_df.dropna()
clean_narrowed_city_df
# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,hamilton,39.1834,-84.5333,26.96,35,0,4.12,US,1695505944
89,89,constantia,44.1833,28.6500,21.50,83,0,4.31,RO,1695506223
183,183,arraial do cabo,-22.9661,-42.0278,21.98,88,0,3.60,BR,1695506250
198,198,mersa matruh,31.3525,27.2453,26.11,78,0,2.57,EG,1695506254
216,216,drama,41.1544,24.1394,24.26,53,0,1.18,GR,1695506259
223,223,saint-pierre,-21.3393,55.4781,20.82,73,0,2.57,RE,1695506261
264,264,chibuto,-24.6867,33.5306,22.98,43,0,3.43,MZ,1695506272
326,326,thohoyandou,-22.9456,30.4850,21.41,30,0,0.87,ZA,1695506290
331,331,spring creek,40.7266,-115.5859,22.67,16,0,3.31,US,1695506291
353,353,santa barbara,34.7333,-120.0343,25.13,52,0,3.60,US,1695506298


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=clean_narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

#Add an empty column, "Hotel Name", so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
47,hamilton,US,39.1834,-84.5333,35,
89,constantia,RO,44.1833,28.6500,83,
183,arraial do cabo,BR,-22.9661,-42.0278,88,
198,mersa matruh,EG,31.3525,27.2453,78,
216,drama,GR,41.1544,24.1394,53,
223,saint-pierre,RE,-21.3393,55.4781,73,
264,chibuto,MZ,-24.6867,33.5306,43,
326,thohoyandou,ZA,-22.9456,30.4850,30,
331,spring creek,US,40.7266,-115.5859,16,
353,santa barbara,US,34.7333,-120.0343,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 20,
    "apiKey":geoapify_key 
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    params["filters"] = f"circle:{Longitude},{Latitude},{radius}"
                 
 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
     
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel : {hotel_df.loc[index, 'Hotel Name']}")
    #print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#hotel_df

Starting hotel search
hamilton - nearest hotel : North Vista Manor
constantia - nearest hotel : Hotel Chérica
arraial do cabo - nearest hotel : No hotel found
mersa matruh - nearest hotel : صقور
drama - nearest hotel : Hydrama Grand Hotel
saint-pierre - nearest hotel : Tropic Hotel
chibuto - nearest hotel : DESHENG
thohoyandou - nearest hotel : Khoroni Hotel Casino and Convention Resort
spring creek - nearest hotel : Ramada
santa barbara - nearest hotel : Fess Parker Wine Country Inn
xai-xai - nearest hotel : Hotel Turístico Halley
beira - nearest hotel : Hotel Savoy
ypsonas - nearest hotel : Club Aphrodite
orchard homes - nearest hotel : Broadway Inn
jackson - nearest hotel : Brookside
pisco - nearest hotel : La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot=hotel_df.hvplot.points(
     "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)